In [1]:
from syft_rds.orchestra import setup_rds_stack
from syft_rds import RDS_NOTEBOOKS_PATH
from syft_rds.models.models import JobStatus, JobUpdate


In [2]:
stack = setup_rds_stack(log_level="INFO")

2025-03-06 15:32:19.017 | WARNING  | syft_rds.orchestra:setup_rds_stack:89 - Using shared data directory for both clients. Any file permission checks will be skipped as both clients have access to the same files.
2025-03-06 15:32:19.021 | INFO     | syft_rds.orchestra:setup_rds_stack:112 - Launching mock RDS stack in /var/folders/6x/3xzhslqj2tx0sgx88x022qyw0000gp/T/rds_dfo85h2r...


In [3]:
do_client = stack.do_rds_client
ds_client = stack.ds_rds_client

# Create a dataset as DO

In [4]:
private_path = RDS_NOTEBOOKS_PATH / "data" / "dataset-1" / "private"
mock_path = RDS_NOTEBOOKS_PATH / "data" / "dataset-1" / "mock"

private_path.mkdir(parents=True, exist_ok=True)
mock_path.mkdir(parents=True, exist_ok=True)

with open(RDS_NOTEBOOKS_PATH /"data/dataset-1/private/data.csv", "w") as f:
    f.write("-1,-2,-3")

with open(RDS_NOTEBOOKS_PATH / "data/dataset-1/mock/data.csv", "w") as f:
    f.write("1,2,3")


In [5]:
data = do_client.dataset.create(
    name="dataset-1",  # MUST BE UNIQUE. Throw Exception if already exist.
    path=private_path,  # MUST EXIST
    mock_path=mock_path,
)

In [6]:
do_client.datasets

[Dataset
   uid: 3d9f2563-b85f-44ac-840f-74e76165c476
   created_at: 2025-03-06T14:32:19.094047Z
   updated_at: 2025-03-06T14:32:19.094052Z
   name: dataset-1
   private: syft://data_owner@test.openmined.org/private/datasets/dataset-1
   mock: syft://data_owner@test.openmined.org/public/datasets/dataset-1
   summary: None
   readme: None
   tags: []]

# Use dataset as DS

In [7]:
ds_client.datasets

[Dataset
   uid: 3d9f2563-b85f-44ac-840f-74e76165c476
   created_at: 2025-03-06T14:32:19.094047Z
   updated_at: 2025-03-06T14:32:19.094052Z
   name: dataset-1
   private: syft://data_owner@test.openmined.org/private/datasets/dataset-1
   mock: syft://data_owner@test.openmined.org/public/datasets/dataset-1
   summary: None
   readme: None
   tags: []]

In [8]:
ds_client.dataset.get("dataset-1").get_private_path()

PosixPath('/private/var/folders/6x/3xzhslqj2tx0sgx88x022qyw0000gp/T/rds_dfo85h2r/shared_client_dir/datasites/data_owner@test.openmined.org/private/datasets/dataset-1')

In [9]:
dataset = ds_client.dataset.get("dataset-1")
dataset.describe()

└── data.csv


True

In [21]:
my_file = RDS_NOTEBOOKS_PATH / "data" / "code2.py"

dataset.set_env()


2025-03-06 15:32:49.785 | INFO     | syft_rds.models.models:set_env:243 - Set SYFT_RDS_DATA_DIR to /private/var/folders/6x/3xzhslqj2tx0sgx88x022qyw0000gp/T/rds_dfo85h2r/shared_client_dir/datasites/data_owner@test.openmined.org/public/datasets/dataset-1 as mock=True


In [25]:
%%writefile {my_file}

import os

DATA_DIR = os.environ["SYFT_RDS_DATA_DIR"] 
OUTPUT_DIR = os.environ.get("SYFT_RDS_OUTPUT_DIR", ".")

print("Hello, world!")

with open(os.path.join(OUTPUT_DIR, "output.txt"), "w") as f:
    f.write("ABC")


Overwriting /Users/azizwork/Workspace/rds/notebooks/data/code2.py


In [26]:
job = ds_client.jobs.submit(
    user_code_path=my_file.as_posix(),
    dataset_name = dataset.name
)
job


Job
  uid: 610b5453-f269-40b4-82b8-5b0b095ad2eb
  created_at: 2025-03-06T14:33:40.975899Z
  updated_at: 2025-03-06T14:33:40.975901Z
  name: crypto_query_jk73
  description: None
  user_code_id: 91b41a00-975e-4ab8-93be-00ac99c59e17
  tags: []
  user_metadata: {}
  status: pending_code_review
  error: no_error
  output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/610b5453-f269-40b4-82b8-5b0b095ad2eb
  dataset_name: dataset-1

# Data Owner runs the job

In [28]:
do_job = do_client.jobs.get_all()[-1]
do_job

Job
  uid: 610b5453-f269-40b4-82b8-5b0b095ad2eb
  created_at: 2025-03-06T14:33:40.975899Z
  updated_at: 2025-03-06T14:33:40.975901Z
  name: crypto_query_jk73
  description: None
  user_code_id: 91b41a00-975e-4ab8-93be-00ac99c59e17
  tags: []
  user_metadata: {}
  status: pending_code_review
  error: no_error
  output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/610b5453-f269-40b4-82b8-5b0b095ad2eb
  dataset_name: dataset-1

In [18]:
do_client.jobs.reject_code(do_job, reason="")

# do_client.jobs.approve_code(do_job) #instead of add_to_queue()


In [16]:
do_job = do_client.run_private(do_job)

2025-03-06 15:32:19.970 | WARNING  | syft_rds.client.rds_client:run_private:119 - Running job without docker is not secure


╭───────────────────────────── Job Configuration ──────────────────────────────╮
│ Starting job                                                                 │
│ Execution: python code2.py                                                   │
│ Dataset Dir.:  .../data_owner@test.openmined.org/private/datasets/dataset-1  │
│ Output Dir.:   .../tmp/syft-rds-jobs/43edaa277ebf4b3f8bcb6fdc8c6e9e30/output │
│ Timeout:  60s                                                                │
╰──────────────────────────────────────────────────────────────────────────────╯

Running job...

Output()

Job completed successfully!

In [17]:
do_job

Job
  uid: 43edaa27-7ebf-4b3f-8bcb-6fdc8c6e9e30
  created_at: 2025-03-06T14:32:19.388093Z
  updated_at: 2025-03-06T14:32:20.233902Z
  name: dynamic_stack_l29e
  description: None
  user_code_id: 6f054520-4113-490c-ab1f-cc122cf1bda5
  tags: []
  user_metadata: {}
  status: job_run_finished
  error: no_error
  output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/43edaa27-7ebf-4b3f-8bcb-6fdc8c6e9e30
  dataset_name: dataset-1

In [ ]:
# do_client.jobs.reject_output(do_job, reason="")
do_client.jobs.share_results(do_job)

2025-03-06 15:32:20.493 | INFO     | syft_rds.client.rds_clients.jobs:share_results:94 - Shared results for job 43edaa27-7ebf-4b3f-8bcb-6fdc8c6e9e30 at /private/var/folders/6x/3xzhslqj2tx0sgx88x022qyw0000gp/T/rds_dfo85h2r/shared_client_dir/datasites/data_owner@test.openmined.org/api_data/RDS/output/43edaa27-7ebf-4b3f-8bcb-6fdc8c6e9e30


(PosixPath('/private/var/folders/6x/3xzhslqj2tx0sgx88x022qyw0000gp/T/rds_dfo85h2r/shared_client_dir/datasites/data_owner@test.openmined.org/api_data/RDS/output/43edaa27-7ebf-4b3f-8bcb-6fdc8c6e9e30'),
 Job
   uid: 43edaa27-7ebf-4b3f-8bcb-6fdc8c6e9e30
   created_at: 2025-03-06T14:32:19.388093Z
   updated_at: 2025-03-06T14:32:20.391946Z
   name: dynamic_stack_l29e
   description: None
   user_code_id: 6f054520-4113-490c-ab1f-cc122cf1bda5
   tags: []
   user_metadata: {}
   status: shared
   error: no_error
   output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/43edaa27-7ebf-4b3f-8bcb-6fdc8c6e9e30
   dataset_name: dataset-1)

In [19]:
import webbrowser

output_path = "/private/var/folders/6x/3xzhslqj2tx0sgx88x022qyw0000gp/T/rds_nn_k_590/shared_client_dir/datasites/data_owner@test.openmined.org/api_data/RDS/output/8de5e43a-94ef-45f8-837c-e114c06a7d2c"
webbrowser.open(f'file://{output_path}')

True